# Análisis viajes - Ecobici

## Objetivo

Comprender los patrones de uso del sistema Eco-bici en la Ciudad de México del mes de agosto de 2024.

## Preguntas a resolver

- ¿Cómo es la distribución de género en el uso de Eco-bici?

- ¿Cómo es la distribución de edad de los usuarios de Eco-bici?

- ¿Cuál es el tiempo promedio de los viajes?

- ¿Qué relación existe entre la duración del viaje y la edad del usuario?

- ¿Cuáles son los horarios y días con mayor cantidad de viajes?

- ¿Cuáles son las ciclo estaciones de retiro y arribo más populares?

- ¿Existen diferencias significativas en el comportamiento entre usuarios de diferentes géneros?

- ¿Qué patrones existen en la relación entre los clusters de retiro y arribo?


## Dependencias

In [1]:
import pandas as pd
import altair as alt

# Datos comúnes
orden_dias = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

## Cargando datos

In [2]:
viajes_path = "viajes_data_limpia.csv"

df_viajes = pd.read_csv(viajes_path)

### Visualización inicial

In [3]:
df_viajes.head(2)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Fecha_Retiro_DateTime,Fecha_Arribo_DateTime,Grupo_Edad,Duracion_viaje_minutos,Duracion_viaje,Hora_Retiro_Digito,Dia_Semana_Retiro,Dia_Retiro_Digito,Hora_Arribo_Digito,Dia_Semana_Arribo,Dia_Arribo_Digito,Cluster_Retiro,Cluster_Arribo
0,O,53.0,2199032,331,312,2024-08-01 00:01:50,2024-08-01 00:04:26,45-54,2.6,02 minutos 36 segundos,0,Thursday,1,0,Thursday,1,3,3
1,F,20.0,3530268,104,055,2024-08-01 00:00:06,2024-08-01 00:05:00,18-23,4.9,04 minutos 54 segundos,0,Thursday,1,0,Thursday,1,4,4


## Análisis - Distribución de usuarios

### Distribución por Género

In [4]:
df_distribucion_por_genero = df_viajes.groupby('Genero_Usuario').size().reset_index(name='Frecuencia')

# Calcular el porcentaje de cada fila
distribucion_por_genero_frecuencia_total = df_distribucion_por_genero['Frecuencia'].sum()
df_distribucion_por_genero['Porcentaje'] = (df_distribucion_por_genero['Frecuencia'] / distribucion_por_genero_frecuencia_total)

#### Visualización

In [5]:
'''
Genero_Usuario
Femenino = F
Masculino = M
Otro/prefiero no decir = O
'''

grafica_distribucion_por_genero = alt.Chart(df_distribucion_por_genero).mark_bar().encode(
      x=alt.X('Genero_Usuario:N', title='Género', sort="-y"),
      y=alt.Y('Frecuencia:Q', title='Frecuencia'),
      color='Genero_Usuario:N',
      tooltip=[
          alt.Tooltip('Genero_Usuario:N', title='Género'),
          alt.Tooltip('Frecuencia:Q', title='Frecuencia', format=','),
          alt.Tooltip('Porcentaje:Q', title='Porcentaje', format='.2%'),
          ]
    ).properties(
        title=f'Distribución de Género ({distribucion_por_genero_frecuencia_total:,} registros)',
        width=500,
        height=300
    ).interactive()

grafica_distribucion_por_genero


alt.Chart(...)

#### Estadísticas

In [6]:
print(df_distribucion_por_genero)

  Genero_Usuario  Frecuencia  Porcentaje
0              F      528908    0.287000
1              M     1277949    0.693451
2              O       36025    0.019548


### Distribución por edad

In [7]:
# Total de registros
edad_total = df_viajes['Edad_Usuario'].count()

# Conteo + Porcentaje
df_distribucion_por_edad = df_viajes['Edad_Usuario'].value_counts().reset_index().astype(int)
df_distribucion_por_edad.columns = ['Edad_Usuario', 'Frecuencia']
df_distribucion_por_edad['Porcentaje'] = df_distribucion_por_edad['Frecuencia'] / edad_total

#### Visualización

In [8]:
alt.Chart(df_distribucion_por_edad).mark_bar().encode(
    alt.X('Edad_Usuario:N', title='Edad', axis=None),
    y='Frecuencia:Q',
    color='Edad_Usuario:N',
    tooltip=[
        alt.Tooltip('Edad_Usuario', title='Edad'),
        alt.Tooltip('Frecuencia:Q', title='Frecuencia', format=','),
        alt.Tooltip('Porcentaje:Q', title='Porcentaje', format='.2%'),
    ]
).properties(
    title='Distribución de edad',
    width=600
).interactive()

alt.Chart(...)

#### Estadísticas

In [9]:
edad_max = df_viajes['Edad_Usuario'].max()
edad_min = df_viajes['Edad_Usuario'].min()
edad_promedio = df_viajes['Edad_Usuario'].mean()
edad_moda = df_viajes['Edad_Usuario'].mode()
edad_mediana = df_viajes['Edad_Usuario'].median()

print(f"Edad máxima: {edad_max} | Edad mínima: {edad_min} | Edad promedio: {edad_promedio} | Edad mediana: {edad_mediana}")

Edad máxima: 104.0 | Edad mínima: 16.0 | Edad promedio: 33.63371013445245 | Edad mediana: 32.0


### Distribución por Grupos de edad

In [10]:
'''
Grupo_Edad
-18
18-23
24-34
35-44
45-54
55-64
65-74
75-84
85-94
95+
'''

# Total de registros
grupos_edad_total = df_viajes['Grupo_Edad'].count()

# Conteo + Porcentaje
df_distribucion_por_grupo_edad = df_viajes['Grupo_Edad'].value_counts().reset_index()
df_distribucion_por_grupo_edad.columns = ['Grupo_Edad', 'Frecuencia']
df_distribucion_por_grupo_edad['Porcentaje'] = df_distribucion_por_grupo_edad['Frecuencia'] / grupos_edad_total


#### Visualización

In [11]:

# Gráfico para grupos de edad mayores al 1%
df_distribucion_por_grupo_edad_mayor_1 = df_distribucion_por_grupo_edad[df_distribucion_por_grupo_edad['Porcentaje'] > 0.01]

grafica_distribucion_por_grupo_edad_mayor_1 = alt.Chart(df_distribucion_por_grupo_edad_mayor_1).mark_bar().encode(
    x=alt.X('Grupo_Edad:N', title='Grupo de Edad'),
    y=alt.Y('Frecuencia:Q', title='Frecuencia'),
    color='Grupo_Edad:N',
    tooltip=[
        alt.Tooltip('Grupo_Edad:N', title='Grupo de edad'),
        alt.Tooltip('Frecuencia:Q', title='Frecuencia', format=','),
        alt.Tooltip('Porcentaje:Q', title='Porcentaje', format='.3%'),
    ]
).properties(
    title='Distribución por grupo de edad mayor al 1%',
    width=400,
    height=200
).interactive()

# Gráfico para grupos de edad menores o igual a 1%
df_distribucion_por_grupo_edad_menor_igual_1 = df_distribucion_por_grupo_edad[df_distribucion_por_grupo_edad['Porcentaje'] <= 0.01]

grafica_distribucion_por_grupo_edad_menor_igual_1 = alt.Chart(df_distribucion_por_grupo_edad_menor_igual_1).mark_bar().encode(
    x=alt.X('Grupo_Edad:N', title='Grupo de Edad'),
    y=alt.Y('Frecuencia:Q', title='Frecuencia'),
    color='Grupo_Edad:N',
    tooltip=[
        alt.Tooltip('Grupo_Edad:N', title='Grupo de edad'),
        alt.Tooltip('Frecuencia:Q', title='Frecuencia', format=','),
        alt.Tooltip('Porcentaje:Q', title='Porcentaje', format='.3%'),
    ]
).properties(
    title='Distribución por grupo de edad menor igual al 1%',
    width=400,
    height=200
).interactive()

# Mostrar gráfico

grafica_distribucion_por_grupo_edad_mayor_1 | grafica_distribucion_por_grupo_edad_menor_igual_1

alt.HConcatChart(...)

#### Estadísticas

In [12]:
df_grupo_edad_populares = df_viajes[df_viajes['Grupo_Edad'].isin(['24-34', '35-44'])]
grupo_edad_populares_total = len(df_grupo_edad_populares)

df_conteo_genero = df_grupo_edad_populares['Genero_Usuario'].value_counts()

print("Distribución de viajes por grupo de edad")
print(df_distribucion_por_grupo_edad)

print("\nDistribución genero por grupo de edad: ['24-34', '35-44']")
print(df_conteo_genero)


Distribución de viajes por grupo de edad
  Grupo_Edad  Frecuencia  Porcentaje
0      24-34      963499    0.522822
1      35-44      442679    0.240210
2      18-23      191746    0.104047
3      45-54      166957    0.090596
4      55-64       61755    0.033510
5      65-74       13279    0.007206
6        -18        1546    0.000839
7      75-84        1340    0.000727
8        95+          63    0.000034
9      85-94          18    0.000010

Distribución genero por grupo de edad: ['24-34', '35-44']
Genero_Usuario
M    957066
F    419911
O     29201
Name: count, dtype: int64


## Análisis - Duración de viajes (promedio)

### Por grupo de edad

In [13]:
df_duracion_viajes_grupo_edad = df_viajes.groupby('Grupo_Edad').agg(
    Duracion_Promedio=('Duracion_viaje_minutos', 'mean'),
    Numero_Viajes=('Duracion_viaje_minutos', 'count')
).reset_index()


#### Visualización

In [14]:
alt.Chart(df_duracion_viajes_grupo_edad).mark_bar().encode(
    alt.X('Grupo_Edad:N', title='Grupo de Edad'),
    alt.Y('Duracion_Promedio:Q', title='Duración promedio (minutos)'),
    color='Grupo_Edad:N',
    tooltip=[
        alt.Tooltip('Duracion_Promedio:Q', title='Duración promedio (min)', format='.2f'),
        alt.Tooltip('Numero_Viajes:Q', title='Número de viajes', format=",")
    ]
).properties(
    title='Duración promedio de viajes por grupo de edad',
    width=600,
    height=400
).interactive()

alt.Chart(...)

#### Estadísticas

In [15]:
duracion_media_general = df_viajes['Duracion_viaje_minutos'].mean()
print(f"Duración media general de los viajes: {duracion_media_general:.2f} minutos")

duracion_por_grupo_edad = df_viajes.groupby('Grupo_Edad')['Duracion_viaje_minutos'].mean().sort_values(ascending=False)
print("\nDuración media por grupo de edad:")
print(duracion_por_grupo_edad)

duracion_por_genero = df_viajes.groupby('Genero_Usuario')['Duracion_viaje_minutos'].mean().sort_values(ascending=False)
print("\nDuración media por género:")
print(duracion_por_genero)

Duración media general de los viajes: 15.19 minutos

Duración media por grupo de edad:
Grupo_Edad
85-94    21.320370
95+      17.328571
65-74    16.206839
18-23    15.466064
45-54    15.286469
75-84    15.261343
55-64    15.239258
24-34    15.205219
-18      15.055239
35-44    14.983362
Name: Duracion_viaje_minutos, dtype: float64

Duración media por género:
Genero_Usuario
F    15.663789
O    15.346924
M    14.996457
Name: Duracion_viaje_minutos, dtype: float64


### Por grupo de edad y género

In [16]:
df_duracion_media_grupo_edad_genero = df_viajes.groupby(['Grupo_Edad', 'Genero_Usuario'])['Duracion_viaje_minutos'].mean().reset_index()

df_duracion_media_grupo_edad_genero = df_duracion_media_grupo_edad_genero.rename(columns={'Duracion_viaje_minutos': 'Duracion_Media'})


#### Visualización

In [17]:
alt.Chart(df_duracion_media_grupo_edad_genero).mark_bar().encode(
    x='Grupo_Edad:N',
    y='Duracion_Media:Q',
    color='Genero_Usuario:N',
    column='Genero_Usuario:N'
).properties(
    title='Duración media de viajes por grupo de edad y género',
    width=200,
    height=300
)

alt.Chart(...)

#### Estadísticas

In [18]:
duracion_por_dia = df_viajes.groupby('Dia_Semana_Retiro')['Duracion_viaje_minutos'].mean().sort_values(ascending=False)
print("\nDuración media por día de la semana:")
print(duracion_por_dia)

duracion_edad_genero = df_viajes.groupby(['Grupo_Edad', 'Genero_Usuario'])['Duracion_viaje_minutos'].mean().unstack()
print("\nDuración media por grupo de edad y género:")
print(duracion_edad_genero)

def identificar_variaciones(serie, media_general, umbral=0.1):
    variaciones = (serie - media_general) / media_general
    return variaciones[abs(variaciones) > umbral]

print("\nVariaciones notables por grupo de edad:")
print(identificar_variaciones(duracion_por_grupo_edad, duracion_media_general))

print("\nVariaciones notables por género:")
print(identificar_variaciones(duracion_por_genero, duracion_media_general))

print("\nVariaciones notables por día de la semana:")
print(identificar_variaciones(duracion_por_dia, duracion_media_general))


Duración media por día de la semana:
Dia_Semana_Retiro
Sunday       19.114335
Saturday     15.332749
Friday       14.909248
Thursday     14.848498
Tuesday      14.791637
Wednesday    14.651454
Monday       14.455232
Name: Duracion_viaje_minutos, dtype: float64

Duración media por grupo de edad y género:
Genero_Usuario          F          M          O
Grupo_Edad                                     
-18             14.432083  15.138203  35.741667
18-23           15.871548  15.300304  15.003129
24-34           15.586874  15.034974  14.773212
35-44           15.550436  14.728770  16.920241
45-54           16.057122  15.079163  14.754182
55-64           15.785461  15.098658  15.278940
65-74           19.438715  15.502754  22.027381
75-84           16.530797  15.260739  10.694444
85-94           23.858974  14.720000        NaN
95+             14.195333  19.233333  19.630000

Variaciones notables por grupo de edad:
Grupo_Edad
85-94    0.403133
95+      0.140425
Name: Duracion_viaje_minutos, 

### Por día de la semana

In [19]:
df_duracion_promedio_dia= df_viajes.groupby('Dia_Semana_Retiro')['Duracion_viaje_minutos'].mean().reset_index()

#### Visualización

In [20]:
alt.Chart(df_duracion_promedio_dia).mark_bar().encode(
    x=alt.X('Dia_Semana_Retiro:N', sort=orden_dias),
    y='Duracion_viaje_minutos:Q',
    color=alt.Color('Dia_Semana_Retiro:N', legend=None),
    tooltip=[
        alt.Tooltip('Dia_Semana_Retiro:N', title='Día'),
        alt.Tooltip('Duracion_viaje_minutos:Q', title='Duracion media', format='.2f')
    ]
).properties(
    title='Duración media de viajes por día de la semana',
    width=400,
    height=300
)

alt.Chart(...)

#### Estadísticas

In [21]:
duracion_por_dia = df_viajes.groupby('Dia_Semana_Retiro')['Duracion_viaje_minutos'].mean().sort_values(ascending=False)
print("\nDuración media por día de la semana:")
print(duracion_por_dia)


Duración media por día de la semana:
Dia_Semana_Retiro
Sunday       19.114335
Saturday     15.332749
Friday       14.909248
Thursday     14.848498
Tuesday      14.791637
Wednesday    14.651454
Monday       14.455232
Name: Duracion_viaje_minutos, dtype: float64


### Por hora del día

In [22]:
df_duracion_hora_retiro = df_viajes.groupby('Hora_Retiro_Digito')['Duracion_viaje_minutos'].mean().reset_index()


#### Visualización

In [23]:
alt.Chart(df_duracion_hora_retiro).mark_line(point=True).encode(
     x=alt.X('Hora_Retiro_Digito:O', title='Hora'),
    y=alt.Y('Duracion_viaje_minutos:Q', title='Duración promedio'),

    tooltip=[
        alt.Tooltip('Hora_Retiro_Digito:N', title='Hora'),
        alt.Tooltip('Duracion_viaje_minutos:Q', title='Duración promedio', format=".2f")
    ]

).properties(
    title='Duración promedio de viajes por hora del día (retiro)',
    width=400,
    height=200
).interactive()

alt.Chart(...)

#### Estadísticas

In [24]:
print(df_duracion_hora_retiro)

    Hora_Retiro_Digito  Duracion_viaje_minutos
0                    0               16.952724
1                    5               11.291183
2                    6               11.945954
3                    7               13.540456
4                    8               14.670781
5                    9               14.445774
6                   10               15.406330
7                   11               16.184140
8                   12               16.143730
9                   13               15.469134
10                  14               15.067005
11                  15               15.099933
12                  16               15.214309
13                  17               16.092596
14                  18               16.915669
15                  19               15.958388
16                  20               15.164087
17                  21               14.424253
18                  22               14.047332
19                  23               14.115685


## Análisis - Cantidad de viajes

### Por día de la semana

In [25]:
df_cantidad_viajes_dia_retiros = df_viajes.groupby(
    'Dia_Semana_Retiro'
).size().reset_index(
    name='Frecuencia'
).sort_values('Frecuencia', ascending= False)


#### Visualización

In [26]:
alt.Chart(df_cantidad_viajes_dia_retiros).mark_bar().encode(
    x=alt.X('Dia_Semana_Retiro:O', sort=orden_dias, title='Día'),
    y=alt.Y('Frecuencia:Q', title='Frecuencia'),
    color='Dia_Semana_Retiro:N',
    tooltip=[
        alt.Tooltip('Dia_Semana_Retiro:N', title='Día'),
        alt.Tooltip('Frecuencia:Q', title='Frecuencia', format=",")
    ]
).properties(
    title='Cantidad de viajes por día de la semana',
    width=400,
    height=300
)


alt.Chart(...)

#### Estadísticas

In [27]:
print(df_cantidad_viajes_dia_retiros)

  Dia_Semana_Retiro  Frecuencia
4          Thursday      359633
0            Friday      312368
5           Tuesday      290632
1            Monday      275232
6         Wednesday      267223
2          Saturday      170372
3            Sunday      167422


### Por hora del día

In [28]:
# Retiro
df_cantidad_viajes_hora_retiros = df_viajes.groupby(
    'Hora_Retiro_Digito'
).size().reset_index(
    name='Frecuencia'
).sort_values('Hora_Retiro_Digito', ascending= False)


#### Visualización

In [29]:
alt.Chart(df_cantidad_viajes_hora_retiros).mark_line(point=True).encode(
     x=alt.X('Hora_Retiro_Digito:O', title='Hora'),
    y=alt.Y('Frecuencia:Q', title='Frecuencia'),

    tooltip=[
        alt.Tooltip('Hora_Retiro_Digito:N', title='Hora'),
        alt.Tooltip('Frecuencia:Q', title='Frecuencia', format=",")
    ]
).properties(
    title='Número de viajes por hora del día (retiro)',
    width=400,
    height=300
).interactive()


alt.Chart(...)

#### Estadísticas

In [30]:
print(df_cantidad_viajes_hora_retiros)

    Hora_Retiro_Digito  Frecuencia
19                  23       22792
18                  22       36626
17                  21       58108
16                  20       79857
15                  19      116825
14                  18      149596
13                  17      133607
12                  16      116253
11                  15      134468
10                  14      138083
9                   13      117617
8                   12       99600
7                   11       92507
6                   10       95071
5                    9      115459
4                    8      153061
3                    7      109024
2                    6       52599
1                    5       14699
0                    0        7030


### Por día - hora

In [31]:
df_viajes_por_dia_hora_retiros = df_viajes.groupby([
    'Dia_Retiro_Digito',
    'Hora_Retiro_Digito'
]).size().reset_index(name='Cantidad_Viajes')

#### Visualización

In [32]:
alt.Chart(df_viajes_por_dia_hora_retiros).mark_rect().encode(
    x=alt.X('Hora_Retiro_Digito:O', title='Hora del día'),
    y=alt.Y('Dia_Retiro_Digito:O', sort=orden_dias, title='Día de la semana'),
    color=alt.Color('Cantidad_Viajes:Q', scale=alt.Scale(scheme='viridis'), title='Cantidad de viajes'),
    tooltip=['Dia_Retiro_Digito', 'Hora_Retiro_Digito', 'Cantidad_Viajes']
).properties(
    title='Cantidad de viajes por día y hora',
    width=600,
    height=400
)

alt.Chart(...)

#### Estadísticas

In [33]:
total_viajes_retiros = df_viajes_por_dia_hora_retiros['Cantidad_Viajes'].sum()
promedio_diario_retiros = total_viajes_retiros / 7  # semana completa
mas_ocupado_retiros = df_viajes_por_dia_hora_retiros.loc[df_viajes_por_dia_hora_retiros['Cantidad_Viajes'].idxmax()]
menos_ocupado_retiros = df_viajes_por_dia_hora_retiros.loc[df_viajes_por_dia_hora_retiros['Cantidad_Viajes'].idxmin()]

print("-" * 80)
print(f"Total de viajes: {total_viajes_retiros:,}")
print(f"Promedio diario de viajes: {promedio_diario_retiros:.2f}")
print(f"Hora más ocupada: {mas_ocupado_retiros['Hora_Retiro_Digito']}:00 con {mas_ocupado_retiros['Cantidad_Viajes']:,} viajes")
print(f"Día más ocupado: el día {mas_ocupado_retiros['Dia_Retiro_Digito']} de agosto 2024")
print(f"Hora menos ocupada: {menos_ocupado_retiros['Hora_Retiro_Digito']}:00 con {menos_ocupado_retiros['Cantidad_Viajes']} viajes")
print(f"Día menos ocupado: el día {menos_ocupado_retiros['Dia_Retiro_Digito']} de agosto 2024")
print("-" * 80)

--------------------------------------------------------------------------------
Total de viajes: 1,842,882
Promedio diario de viajes: 263268.86
Hora más ocupada: 18:00 con 7,797 viajes
Día más ocupado: el día 19 de agosto 2024
Hora menos ocupada: 0:00 con 70 viajes
Día menos ocupado: el día 7 de agosto 2024
--------------------------------------------------------------------------------


## Análisis - Ciclo estaciones

### Top 20

In [34]:
# Retiros
df_ciclo_estacion_retiros = df_viajes['Ciclo_Estacion_Retiro'].value_counts().reset_index()
df_ciclo_estacion_retiros.columns = ['Ciclo_Estacion', 'Cantidad_Retiros']

# Arribos
df_ciclo_estacion_arribos = df_viajes['Ciclo_Estacion_Arribo'].value_counts().reset_index()
df_ciclo_estacion_arribos.columns = ['Ciclo_Estacion', 'Cantidad_Arribos']

# Combinamos retiros y arribos
df_ciclo_estacion_populares = pd.merge(df_ciclo_estacion_retiros, df_ciclo_estacion_arribos, on='Ciclo_Estacion', how='outer').fillna(0)
df_ciclo_estacion_populares['Total_Viajes'] = df_ciclo_estacion_populares['Cantidad_Retiros'] + df_ciclo_estacion_populares['Cantidad_Arribos']

# Ordenamos por total de viajes
df_ciclo_estacion_populares = df_ciclo_estacion_populares.sort_values('Total_Viajes', ascending=False).reset_index(drop=True)

# Tomamos las 20 estaciones más populares para la visualización
df_top_20_ciclo_estaciones = df_ciclo_estacion_populares.head(20)

df_top_20_ciclo_estaciones.head(1)



,Ciclo_Estacion,Cantidad_Retiros,Cantidad_Arribos,Total_Viajes
0,271-272,22035,30013,52048


#### Visualización

In [35]:
ciclo_estaciones_data = pd.melt(
    df_top_20_ciclo_estaciones,
    id_vars=['Ciclo_Estacion'],
    value_vars=['Cantidad_Retiros', 'Cantidad_Arribos']
)

ciclo_estaciones_data.columns = ['Ciclo_Estacion', 'Tipo', 'Cantidad']

grafica_ciclo_estaciones_populares = alt.Chart(ciclo_estaciones_data).mark_bar().encode(
    x=alt.X('Cantidad:Q', title='Cantidad de viajes'),
    y=alt.Y('Ciclo_Estacion:N', sort='-x', title='Ciclo estación'),
    color=alt.Color(
        'Tipo:N',
        scale=alt.Scale(domain=['Cantidad_Retiros', 'Cantidad_Arribos'], range=['#1f77b4', '#ff7f0e'])),
    tooltip=[
        alt.Tooltip('Ciclo_Estacion:N', title='C. Estación'),
        alt.Tooltip('Cantidad:Q', title='Cantidad', format=","),
        alt.Tooltip('Tipo:N', title='Tipo'),
    ]
).properties(
    title='Top 20 Estaciones más populares',
    width=600,
    height=400
)


grafica_ciclo_estaciones_populares


alt.Chart(...)

#### Estadísticas

In [36]:
print("Estadísticas de las estaciones más populares:")
print(df_top_20_ciclo_estaciones[['Cantidad_Retiros', 'Cantidad_Arribos', 'Total_Viajes']].describe())

print("\nTop 5 estaciones de retiro:")
print(df_top_20_ciclo_estaciones[['Ciclo_Estacion', 'Cantidad_Retiros']].head(5))

print("\nTop 5 estaciones de arribo:")
print(df_top_20_ciclo_estaciones[['Ciclo_Estacion', 'Cantidad_Arribos']].head(5))



Estadísticas de las estaciones más populares:
       Cantidad_Retiros  Cantidad_Arribos  Total_Viajes
count         20.000000         20.000000      20.00000
mean        9202.200000       9987.100000   19189.30000
std         3167.722453       5057.873944    8118.60154
min         6930.000000       6624.000000   14826.00000
25%         8010.250000       7895.500000   15878.00000
50%         8413.500000       8742.500000   17044.00000
75%         9036.500000       9151.750000   17707.75000
max        22035.000000      30013.000000   52048.00000

Top 5 estaciones de retiro:
  Ciclo_Estacion  Cantidad_Retiros
0        271-272             22035
1            014              9089
2            027             10852
3            064             10406
4            018              8619

Top 5 estaciones de arribo:
  Ciclo_Estacion  Cantidad_Arribos
0        271-272             30013
1            014             14722
2            027             12088
3            064             10881
4      

### Clusters de Retiro y Arribo

In [37]:
# Creamos una tabla de contingencia
cluster_matrix = pd.crosstab(df_viajes['Cluster_Retiro'], df_viajes['Cluster_Arribo'])

# Calculamos el porcentaje de viajes para cada combinación de clusters
cluster_percentage = cluster_matrix.div(cluster_matrix.sum().sum()) * 100



#### Visualización

In [38]:
# Preparación
cluster_data = cluster_percentage.reset_index().melt(id_vars='Cluster_Retiro', var_name='Cluster_Arribo', value_name='Porcentaje')


alt.Chart(cluster_data).mark_rect().encode(
    x=alt.X('Cluster_Arribo:N', title='Cluster de Arribo'),
    y=alt.Y('Cluster_Retiro:N', title='Cluster de Retiro'),
    color=alt.Color('Porcentaje:Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['Cluster_Retiro', 'Cluster_Arribo', alt.Tooltip('Porcentaje:Q', format='.2f')]
).properties(
    title='Patrones de viaje entre Clusters de Retiro y Arribo',
    width=500,
    height=400
)

alt.Chart(...)

#### Estadísticas

In [39]:
# Calculamos estadísticas por cluster de retiro
stats_retiro = df_viajes.groupby('Cluster_Retiro').agg({
    'Duracion_viaje_minutos': ['mean', 'median', 'std'],
    'Cluster_Arribo': lambda x: x.mode().iloc[0]  # Cluster de arribo más común
}).round(2)
stats_retiro.columns = ['Duración_Media', 'Duración_Mediana', 'Duración_Std', 'Cluster_Arribo_Común']

# Calculamos estadísticas por cluster de arribo
stats_arribo = df_viajes.groupby('Cluster_Arribo').agg({
    'Duracion_viaje_minutos': ['mean', 'median', 'std'],
    'Cluster_Retiro': lambda x: x.mode().iloc[0]  # Cluster de retiro más común
}).round(2)
stats_arribo.columns = ['Duración_Media', 'Duración_Mediana', 'Duración_Std', 'Cluster_Retiro_Común']

# Imprimimos las estadísticas
print("Estadísticas por Cluster de Retiro:")
print(stats_retiro)
print("\nEstadísticas por Cluster de Arribo:")
print(stats_arribo)
print("\nPorcentaje de viajes entre clusters (top 5):")
print(cluster_percentage.stack().nlargest(5))
print("\nTop 5 combinaciones de clusters más frecuentes:")
print(cluster_matrix.stack().nlargest(5))

Estadísticas por Cluster de Retiro:
                Duración_Media  Duración_Mediana  Duración_Std  \
Cluster_Retiro                                                   
0                        15.27             12.08         25.94   
1                        13.92             11.10         17.26   
2                        13.63             10.45         13.08   
3                        14.62             11.38         15.55   
4                        15.49             12.57         59.36   
5                        17.73             14.03         60.45   

                Cluster_Arribo_Común  
Cluster_Retiro                        
0                                  0  
1                                  1  
2                                  2  
3                                  3  
4                                  4  
5                                  5  

Estadísticas por Cluster de Arribo:
                Duración_Media  Duración_Mediana  Duración_Std  \
Cluster_Arribo      

## Conclusiones

### Distribución de usuarios

- La **mayoría** de los usuarios son **hombres**, representando el **69.35%** de los viajes, mientras que las **mujeres** constituyen el **28.70%.**
- Un pequeño porcentaje de los usuarios no especificó su género **(1.95%)**.
- El grupo de personas predominante es el de **24-34 años**, que representa más del **52%** de los viajes. Le sigue el grupo de **35-44 años** con un **24%**, mientras que el grupo de **18-23 años** realiza aproximadamente el **10%** de los viajes.
- La **edad** promedio de los usuarios es de **33.63 años** y la mediana es de **32 años**.
- Los usuarios de mayor edad (mayores de 65 años) realizan un porcentaje pequeño de los viajes, destacando que el grupo de 95 años o más solo realizó 63 viajes.


### Duración de viajes

- Las **mujeres** tienen una duración media ligeramente superior (15.66 minutos) en comparación con los **hombres** (14.99 minutos).
- Los usuarios que no especifica su género tienen una duración media de 15.35 minutos.
- La duración media de los viajes aumenta con la edad, alcanzando su máximo en el grupo de **85-94 años** con 21.32 minutos, mientras que el grupo de **35-44 años** tiene la duración más baja, de 14.98 minutos.
- Los viajes realizados los **domingos** tienen la duración más alta, con un promedio de 19.11 minutos, lo que sugiere que los usuarios podrían usar las bicicletas de manera recreativa.
- Los **lunes** tienen la duración más baja, con una media de 14.45 minutos, lo que podría estar relacionado con viajes más cortos y enfocados en el transporte cotidiano.
- La duración más **larga** se registra a las 12:00 AM (medianoche), con 16.95 minutos, seguida de las 6:00 PM con 16.92 minutos. Esto podría reflejar la relajación de los usuarios en horas no laborales.
- Las duraciones más **cortas** ocurren a las 5:00 AM, con un promedio de 11.29 minutos, posiblemente por usuarios en tránsito a sus trabajos.

**Patrones generales**

- Los viajes son más cortos durante los días laborales y en las primeras horas de la mañana, sugiriendo un uso enfocado en la movilidad urbana cotidiana.
- Los fines de semana, especialmente los domingos, se caracterizan por viajes más largos, posiblemente relacionados con actividades recreativas.

### Cantidad de viajes

- El día **jueves** es el más popular para realizar viajes, con 359,633 viajes, seguido por el viernes (312,368 viajes) y el martes (290,632 viajes).
- Los días con menos viajes son los **fines de semana**, siendo el sábado y domingo los menos ocupados, con 170,372 y 167,422 viajes, respectivamente. Esto puede sugerir que el uso de Eco-bici está más relacionado con actividades de transporte diario que con el ocio.
- La hora pico es las 18:00 horas, con 149,596 viajes, lo que indica que la mayor demanda de bicicletas ocurre al final de la jornada laboral, durante el regreso a casa.
- La hora menos ocupada es la medianoche (00:00 horas), con solo 7,030 viajes.
- Las horas de la mañana, especialmente entre las 8:00 y 9:00 horas, también muestran un aumento en la actividad, lo que indica que Eco-bici es una opción popular para el desplazamiento al trabajo o escuela.

### Ciclo estaciones

- La estación **271-272** (CE-271-272 Jesús García - Carlos J. Meneses, Buenavista) es la más popular tanto para retiros (22,035 viajes) como para arribos (30,013 viajes), lo que la convierte en el principal hub de Eco-bici. Otras estaciones populares incluyen las 014 (CE-014 Reforma - Río de Plata), 027 (CE-027 Reforma- Havre), y 064 (CE-064 Sonora - Ámsterdam).
- Los clusters de retiro y arribo presentan una duración media de los viajes relativamente consistente, oscilando entre 13.63 y 17.73 minutos.
- Los clusters con mayor duración de viajes son el Cluster 5, tanto para retiro como para arribo, con duraciones medias de 17.73 minutos y 16.96 minutos respectivamente.
- Las estaciones agrupadas en el **Cluster 0** presentan la **mayor cantidad de viajes** tanto en retiros como en arribos, lo que sugiere que esta área tiene una alta concentración de usuarios.
- El **19.32%** de los viajes ocurren entre estaciones dentro del **Cluster 0**, lo que lo convierte en el patrón de viaje más común.
- Existen también patrones frecuentes entre Cluster 0 y Cluster 4 (7.27%) y viceversa (7.03%), lo que indica conexiones regulares entre estas áreas específicas de la ciudad.

El análisis de los clusters revela que la mayoría de los usuarios **no recorren largas distancias** entre estaciones de retiro y arribo, sino que tienden a permanecer dentro del mismo cluster o áreas cercanas.

- El **19 de agosto de 2024** fue el día más ocupado, lo que puede haber sido impulsado por un evento o un aumento general en la demanda durante esa fecha.
- El día menos ocupado fue el **7 de agosto de 2024**, lo que podría estar relacionado con factores externos como el clima o circunstancias locales.

**Sugerencias**

- Las estaciones más populares, especialmente la 271-272 (CE-271-272 Jesús García - Carlos J. Meneses, Buenavista), requieren atención en cuanto a la reposición de bicicletas para evitar problemas de disponibilidad durante las horas pico.
- La alta concentración de viajes dentro de ciertos clusters puede sugerir áreas clave para mejorar la infraestructura ciclista o expandir el sistema a nuevas zonas.
- El análisis de los horarios y días más populares puede ayudar a mejorar la logística del sistema y planificar mejor los mantenimientos en horas de menor uso.

## En construcción

### Cluster 0 (más concurrido)

In [55]:
df_viajes_cluster_0 = df_viajes.query('Cluster_Retiro == 0 and Cluster_Arribo == 0 and \
                 8 <= Hora_Retiro_Digito <= 18 and 8 <= Hora_Arribo_Digito <= 18 and \
                 Dia_Semana_Retiro in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] and \
                 Dia_Semana_Arribo in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]')


In [57]:
df_prueba = df_viajes_cluster_0.groupby([
    'Ciclo_Estacion_Retiro',
    'Ciclo_Estacion_Arribo'
]).size().reset_index(name='Frecuencia').sort_values('Frecuencia', ascending= False)


In [63]:
df_prueba.shape

(20781, 3)

In [64]:
alt.Chart(df_prueba.iloc[0:5000]).mark_rect().encode(
    x=alt.X('Ciclo_Estacion_Retiro:O', axis=None),
    y=alt.Y('Ciclo_Estacion_Arribo:O', axis=None),
    color='Frecuencia:Q',
    tooltip=[
        alt.Tooltip('Ciclo_Estacion_Retiro:O', title='C. Estación retiro'),
        alt.Tooltip('Ciclo_Estacion_Arribo:O', title='C. Estación arribo'),
        alt.Tooltip('Frecuencia:Q', title='Frecuencia', format=",")
    ]
).properties(
    title='Flujo de viajes entre ciclo estaciones (Cluster 0)',
    width=500,
    height=500
).interactive()

alt.Chart(...)